In [1]:
import plotly.express as px
import pandas as pd
from psycopg2 import connect

from dash import Dash
from dash import dash_table, html, dcc
from dash import callback, Input, Output

 ## Konfiguracja połączenia


In [2]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airline'
port = 5432

In [3]:
con = connect(
    user=username,
    password=password,
    host=host,
    database=database,
    port=port
)

cursor = con.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`


In [4]:
top_routes_df = pd.read_sql(
    'select * from reporting.top_reliability_roads',
    con=con
)
top_routes_df.head()

/tmp/ipykernel_10263/613399551.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_routes_df = pd.read_sql(


origin_airport_id                         origin_airport_name  \
0              13930  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US   
1              12953                    LAGUARDIA AIRPORT, NY US   
2              12892    LOS ANGELES INTERNATIONAL AIRPORT, CA US   
3              14771  SAN FRANCISCO INTERNATIONAL AIRPORT, CA US   
4              12892    LOS ANGELES INTERNATIONAL AIRPORT, CA US   

   dest_airport_id                           dest_airport_name  year  \
0            12953                    LAGUARDIA AIRPORT, NY US  2019   
1            13930  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  2019   
2            14771  SAN FRANCISCO INTERNATIONAL AIRPORT, CA US  2019   
3            12892    LOS ANGELES INTERNATIONAL AIRPORT, CA US  2019   
4            12478                    LAGUARDIA AIRPORT, NY US  2019   

   flights_amount  reliability  nb  
0           14219     0.274140   1  
1           14203     0.238964   2  
2           14096     0.234109   3  
3           14088     0.199886   4  
4           12797     0.194499   5

In [5]:
top_routes_df['reliability'] = (
    top_routes_df['reliability'].apply(
        lambda x: '{:.2f}%'.format(x*100)
    )
)

In [6]:
columns = {
    'origin_airport_name': 'Origin',
    'dest_airport_name': 'Destination',
    'year': 'Year',
    'reliability': 'Reliability',
    'nb': 'Rank'
}
top_routes_df.rename(columns=columns, inplace=True)


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 

In [7]:
yoy_comparision_df = pd.read_sql(
    sql='select * from reporting.year_to_year_comparision',
    con=con
)

/tmp/ipykernel_10263/1228231779.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yoy_comparision_df = pd.read_sql(


In [8]:
yoy_comparision_to_plot_df = yoy_comparision_df.pivot(
    index='month',
    columns='year',
    values='reliability'
)

In [9]:
day_by_day_reliability_fig_labels = {
    "month": "Month",
    "value": "Reliability",
    "variable": "Year"
}

yoy_comparision_fig = px.bar(
    yoy_comparision_to_plot_df,
    x=yoy_comparision_to_plot_df.index,
    y=[2019, 2020],
    barmode="group",
    labels=day_by_day_reliability_fig_labels,
    title="YoY comparision of flight reliability",
)
yoy_comparision_fig.update_layout(hovermode="x")
yoy_comparision_fig.update_xaxes(type='category')

yoy_comparision_fig.update_yaxes(tickformat='.0%', dtick=0.05)
yoy_comparision_fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(1, 13)),
    ticktext=['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']
)

 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020


In [10]:
day_to_day_comparision_df = pd.read_sql(
    sql='select * from reporting.day_to_day_comparision',
    con=con
)

/tmp/ipykernel_10263/1486272750.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [11]:
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot(
    index='day_of_week',
    columns='year',
    values='reliability'
)

In [12]:
day_by_day_reliability_fig_labels = {
    "month": "Month",
    "value": "Reliability",
    "variable": "Year"
}

day_to_day_comparision_fig = px.bar(
    day_to_day_comparision_to_plot_df,
    x=day_to_day_comparision_to_plot_df.index,
    y=[2019, 2020],
    barmode="group",
    labels=day_by_day_reliability_fig_labels,
    title="WKoWK comparision of flight reliability",
)
day_to_day_comparision_fig.update_layout(hovermode="x")
day_to_day_comparision_fig.update_xaxes(type='category')

day_to_day_comparision_fig.update_yaxes(tickformat='.0%', dtick=0.05)
day_to_day_comparision_fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(1, 8)),
    ticktext=['Monday', 'Tuesday', 'Wednesday',
              'Thursday', 'Friday', 'Saturday', 'Sunday']
)

 # Stworzenie szeregu czasowego
 

In [13]:
day_by_day_reliability_df = pd.read_sql(
    sql='select * from reporting.day_by_day_reliability',
    con=con,
    parse_dates='flight_date'
)
day_by_day_reliability_df.head()

/tmp/ipykernel_10263/979633216.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



flight_date  reliability
0  2019-01-01     0.186556
1  2019-01-02     0.215106
2  2019-01-03     0.145952
3  2019-01-04     0.114245
4  2019-01-05     0.188235

In [14]:
day_by_day_reliability_fig_labels = {
    "flight_date": "Date",
    "reliability": "Reliability",
    "color": "Year"
}

day_by_day_reliability_fig = px.line(
    day_by_day_reliability_df,
    x='flight_date',
    y='reliability',
    color=day_by_day_reliability_df['flight_date'].dt.year,
    title='Day by day flight reliability',
    labels=day_by_day_reliability_fig_labels
)

day_by_day_reliability_fig.update_yaxes(tickformat='.0%', dtick=0.05)

In [15]:
day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

In [16]:
top_routes_table = dash_table.DataTable(
    id='top-routes',
    data=top_routes_df[columns.values()].to_dict('records'),
)

In [17]:
top_routes_page_title = html.H3(
    "TOP 10 reliability routes in 2019 and 2020",
    style={"fontFamily": "verdana", "color": "#444"}
)

In [18]:
top_routes_page_layout = html.Div(
    children=[
        top_routes_page_title,
        top_routes_table
    ])

 ## Stworzenie layout dla widoku `Comparision`


In [19]:
yoy_comparision_component = dcc.Graph(
    id='yoy_comparision_fig',
    figure=yoy_comparision_fig
)

In [20]:
day_to_day_comparision_component = dcc.Graph(
    id='day_to_day_comparision_fig',
    figure=day_to_day_comparision_fig
)

In [21]:
comparision_layout = html.Div(children=[
    yoy_comparision_component,
    day_to_day_comparision_component
])

 # Konfigurowanie aplikacji


 Tutaj przygotuj layout aplikacji

In [22]:
top_report_link = dcc.Link(html.Button('TOP report'), href='/')
yoy_report_link = dcc.Link(html.Button(
    'Comparision'), href='/comparision_reporting')
dbd_report_link = dcc.Link(html.Button('Day by day reliability'),
                           href='/day_by_day_reporting')

links_layout = html.Div([
    top_report_link,
    yoy_report_link,
    dbd_report_link])

app_layout = [
    dcc.Location(id='url', refresh=False),
    links_layout,
    html.Div(id='page-content')
]

In [23]:
@ callback(
    Output('page-content', 'children'),
    Input('url', 'pathname'))
def display_page(pathname):

    if pathname == "/comparision_reporting":
        return comparision_layout
    elif pathname == '/day_by_day_reporting':
        return day_by_day_layout
    else:
        return top_routes_page_layout